<a href="https://colab.research.google.com/github/RyosukeHanaoka/TechTeacher_New/blob/main/WorksDec_27_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#optPhotoFilesの整備(RA group)
これまでに、1. 299例のRA患者の手背の光学写真を撮影し、2. 被検者の写真と臨床像を照合する表を作成した。次にRmbgを使用して手背以外の背景部分を消去する。

ただし、この作業では数枚の写真において撮影背景に誤って添付されていた付箋を消去できない。これについてはのちにpixelmeterを使用し、手作業で付箋の画像を消去する。画像はpxmで保存せず、jpgに変換して書き出す。

###GoogleDriveをマウントする。

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###背景消去作業(RA_Right)

nonRAのvolunteerで撮影された画像は、非常に雑多な背景のなかに手背の画像が撮影されているため、背景が学習に悪影響を及ぼさないように、背景を削除した。これらの画像と条件を同一にするために、RA患者の画像でも同様の処理を行う。
ライブラリrembgのremoveとnew_sessionを使用する。 背景を削除した画像は、/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/RA/RA_Right_Handに格納する。

In [ ]:
#Rembgをインストールする。
!pip install rembg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.5 MB/s eta 0:00:00


In [ ]:
#必要なライブラリをインポートする。
import os
from rembg import remove
from concurrent.futures import ProcessPoolExecutor

まず、右手の画像を格納した"RA_Right"から背景除去処理を行う。

In [ ]:
#背景の除去を目的とした関数、remove_bg_from_fileを定義する。
def remove_bg_from_file(file_path, dst_dir):
    with open(file_path, "rb") as f:
        img = f.read()
        output = remove(img)

    dst_file_path = os.path.join(dst_dir, os.path.basename(file_path))
    with open(dst_file_path, "wb") as f_out:
        f_out.write(output)

# 画像の背景を削除するディレクトリを指定する。
src_dir = "/content/drive/MyDrive/RA_Right"
dst_dir = "/content/drive/MyDrive/RA/RA_Right_Hand"

# ディレクトリが存在しない場合、新規に作成する。
if not os.path.exists(dst_dir):
    os.makedirs(dst_dir)

file_paths = [os.path.join(src_dir, file) for file in os.listdir(src_dir) if file.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Use all available CPU cores for parallel processing
with ProcessPoolExecutor() as executor:
    executor.map(remove_bg_from_file, file_paths, [dst_dir] * len(file_paths))

print("Background removal and saving to new directory completed!")

100%|███████████████████████████████████████| 176M/176M [00:00<00:00, 11.1GB/s]
100%|███████████████████████████████████████| 176M/176M [00:00<00:00, 28.7GB/s]


Background removal and saving to new directory completed!


In [ ]:
#背景の除去を目的とした関数、remove_bg_from_fileを定義する。
def remove_bg_from_file(file_path, dst_dir):
    with open(file_path, "rb") as f:
        img = f.read()
        output = remove(img)

    dst_file_path = os.path.join(dst_dir, os.path.basename(file_path))
    with open(dst_file_path, "wb") as f_out:
        f_out.write(output)

# 画像の背景を削除するディレクトリを指定する。
src_dir = "/content/drive/MyDrive/RA/need_to_Rmbg"
dst_dir = "/content/drive/MyDrive/RA/RA_Right_Hand"

# ディレクトリが存在しない場合、新規に作成する。
if not os.path.exists(dst_dir):
    os.makedirs(dst_dir)

file_paths = [os.path.join(src_dir, file) for file in os.listdir(src_dir) if file.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Use all available CPU cores for parallel processing
with ProcessPoolExecutor() as executor:
    executor.map(remove_bg_from_file, file_paths, [dst_dir] * len(file_paths))

print("Background removal and saving to new directory completed!")

Background removal and saving to new directory completed!


つぎに、関数"remobe_bg_from_file"を使用して、左手の画像を格納した"RA_Left"にも同様の背景除去処理を行う。

In [ ]:
# 画像の背景を削除するディレクトリを指定する。
src_dir = "/content/drive/MyDrive/RA_Left"
dst_dir = "/content/drive/MyDrive/RA/RA_Left_Hand"

# ディレクトリが存在しない場合、新規に作成する。
if not os.path.exists(dst_dir):
    os.makedirs(dst_dir)

file_paths = [os.path.join(src_dir, file) for file in os.listdir(src_dir) if file.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Use all available CPU cores for parallel processing
with ProcessPoolExecutor() as executor:
    executor.map(remove_bg_from_file, file_paths, [dst_dir] * len(file_paths))

print("Background removal and saving to new directory completed!")

Background removal and saving to new directory completed!


###背景除去処理後も残存する付箋の画像を除去する（pixlmeterを使用）。
当初のco-medical_workerへの指示の不備によって、研究者の意図が伝わらず、右手と左手を簡単に識別するために背景に"R","L"と記載された付箋を貼付してある写真がある。これらの付箋が学習に悪影響を与える可能性を排除するため、画像上から消去する必要があるが、これはRembgでも除去不可能である。そこで、ローカル環境内に一度画像をダウンロードし、pixelmeterによる手作業でこの付箋の画像を除去する必要が生じた。
短時間で作業を終えるため、付箋が添付された1-19までの画像のみダウンロード、作業を終えたら、GoogleDriveにアップロードする。

###問題発生

pixelmeterを使用して付箋の画像を消去、jpgフォーマットで書き出したところ、背景が透明ではなく、白地になってしまった。これでは他の画像と同一の条件とは言い難い。このため、白地を透明に変更するために、もう一度Rmbgにかけることにした。

In [ ]:
#背景の除去を目的とした関数、remove_bg_from_fileを定義する。
def remove_bg_from_file(file_path, dst_dir):
    with open(file_path, "rb") as f:
        img = f.read()
        output = remove(img)

    dst_file_path = os.path.join(dst_dir, os.path.basename(file_path))
    with open(dst_file_path, "wb") as f_out:
        f_out.write(output)

# 画像の背景を削除するディレクトリ、"need_to_Rmbg"を作成、そこに問題の画像を格納。（まず右手）
src_dir = "/content/drive/MyDrive/RA/need_to_Rmbg"
# 作業の終了した画像ファイルは"RA_Right_Hand"に戻す。
dst_dir = "/content/drive/MyDrive/RA/RA_Right_Hand"

# ディレクトリが存在しない場合、新規に作成する。
if not os.path.exists(dst_dir):
    os.makedirs(dst_dir)

file_paths = [os.path.join(src_dir, file) for file in os.listdir(src_dir) if file.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Use all available CPU cores for parallel processing
with ProcessPoolExecutor() as executor:
    executor.map(remove_bg_from_file, file_paths, [dst_dir] * len(file_paths))

print("Background removal and saving to new directory completed!")

Background removal and saving to new directory completed!


In [ ]:
# 画像の背景を削除するディレクトリ、"need_to_Rmbg"を作成、そこに問題の画像を格納。（左手）
src_dir = "/content/drive/MyDrive/RA/need_to_Rmbg"
# 作業の終了した画像ファイルは"RA_Left_Hand"に戻す。
dst_dir = "/content/drive/MyDrive/RA/RA_Left_Hand"

file_paths = [os.path.join(src_dir, file) for file in os.listdir(src_dir) if file.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Use all available CPU cores for parallel processing
with ProcessPoolExecutor() as executor:
    executor.map(remove_bg_from_file, file_paths, [dst_dir] * len(file_paths))

print("Background removal and saving to new directory completed!")

#鏡像の作成
深層学習のサンプル数は可能な限り多い方が良い。手背の画像は左右対称であるため、鏡像を作成して画像の数を増やすことができる。ここではまずRA群、nonRA群ともに左手の画像の鏡像を作成、右手画像と合わせることでサンプル数を倍にする。

まず、RA群の処理を行う。"content/drive/MyDrive/RA/RA_Right_Hand"と"content/drive/MyDrive/RA/RA_Left_Hand"の鏡像を作成、"content/drive/MyDrive/RA/RA_Right_Hand_mirror"と"content/drive/MyDrive/RA/RA_Left_Hand_mirror"と命名する。
さらに、"content/drive/MyDrive/RA/RA_Right_Hand"と、"content/drive/MyDrive/RA/RA_Left_Hand_mirror"を結合させて"optPhotoRaRight"とし、"content/drive/MyDrive/RA/RA_Left_Hand"と、"content/drive/MyDrive/RA/RA_Right_Hand_mirror"を結合させて、"optPhotoRaLeft"と命名する。

###注意点
pngと異なりjpgではPILを使用した際に色彩チャンネルの扱いに相違が存在するため、"img.convert"を使用して色彩チャンネルの扱いを調整する必要があることに注意が必要である。

In [ ]:
from PIL import Image
import os
import shutil
from PIL import Image
import os

def mirror_images(source_dir, dest_dir):
    # Create the destination directory if it doesn't exist
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    # List all jpeg files in the source directory
    for filename in os.listdir(source_dir):
        if filename.endswith(".jpg"):
            # Open the image
            img_path = os.path.join(source_dir, filename)
            with Image.open(img_path) as img:
                # Convert to 'RGB' if 'RGBA'
                if img.mode == 'RGBA':
                    img = img.convert('RGB')

                # Mirror the image
                mirrored_img = img.transpose(Image.FLIP_LEFT_RIGHT)

                # Save the mirrored image to the destination directory
                mirrored_img.save(os.path.join(dest_dir, filename))


def copy_images(source_dirs, dest_dir):
    """
    Copy all images from the source directories to the destination directory.
    """
    # Create the destination directory if it doesn't exist
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    for source_dir in source_dirs:
        for filename in os.listdir(source_dir):
            if filename.endswith(".jpg"):
                src_path = os.path.join(source_dir, filename)
                dest_path = os.path.join(dest_dir, filename)
                # Copy the image
                shutil.copy(src_path, dest_path)

# Directories
left_hand_dir = "/content/drive/MyDrive/RA/RA_Left_Hand"
right_hand_dir = "/content/drive/MyDrive/RA/RA_Right_Hand"

left_hand_mirror_dir = "/content/drive/MyDrive/RA/RA_Left_Hand_mirror"
right_hand_mirror_dir = "/content/drive/MyDrive/RA/RA_Right_Hand_mirror"

opt_photo_ra_right_dir = "/content/drive/MyDrive/OptPhotoFiles/optPhotoRaRight"
opt_photo_ra_left_dir = "/content/drive/MyDrive/OptPhotoFiles/optPhotoRaLeft"

# Mirror images
mirror_images(left_hand_dir, left_hand_mirror_dir)
mirror_images(right_hand_dir, right_hand_mirror_dir)

# Copy images
copy_images([right_hand_dir, left_hand_mirror_dir], opt_photo_ra_right_dir)
copy_images([left_hand_dir, right_hand_mirror_dir], opt_photo_ra_left_dir)

"Images have been processed and saved to the specified directories."

'Images have been processed and saved to the specified directories.'

#ローカル環境へのダウンロード
GoogleDrive上での作業を減らし、作業をできるだけスクリプトとして記録に残すため、"shutil"を使用してGoogle Colab上でファイル圧縮とダウンロードを行うように工夫してみたが、GoogleDrive上での作業よりもはるかに時間がかかることが判明した。蛇足だったと言える。

In [ ]:
import shutil

# 圧縮するディレクトリのパス
directory_to_zip = '/content/drive/MyDrive/OptPhotoFiles/optPhotoRaRight'

# ZIPファイルの出力パス
zip_output = '/content/optPhotoRARight.zip'

# ディレクトリをZIPファイルに圧縮
shutil.make_archive(zip_output[:-4], 'zip', directory_to_zip)

# ZIPファイルをダウンロード
from google.colab import files
files.download(zip_output)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 圧縮するディレクトリのパス
directory_to_zip = '/content/drive/MyDrive/OptPhotoFiles/optPhotoRaLeft'

# ZIPファイルの出力パス
zip_output = '/content/optPhotoRARight.zip'

# ディレクトリをZIPファイルに圧縮
shutil.make_archive(zip_output[:-4], 'zip', directory_to_zip)

# ZIPファイルをダウンロード
from google.colab import files
files.download(zip_output)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#鏡像の作成（nonRA）
これまでnonRAの画像を整理するために、ローカル環境での手作業を続けてきた。画像の向きのばらつき、左右の間違いなど、ファイルの形式ではなく、画像の内容を人間が目視しなければ不可能な作業が続いたためである。
本日をもってそれが終了したため、今後の自動化した作業のためにnonRAの画像一式をGoogleDriveにアップロードした。これを用いてnonRA群の鏡像作成を行う。

In [2]:
from PIL import Image
import os
import shutil
from PIL import Image
import os

def mirror_images(source_dir, dest_dir):
    # Create the destination directory if it doesn't exist
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    # List all jpeg files in the source directory
    for filename in os.listdir(source_dir):
        if filename.endswith(".jpg"):
            # Open the image
            img_path = os.path.join(source_dir, filename)
            with Image.open(img_path) as img:
                # Convert to 'RGB' if 'RGBA'
                if img.mode == 'RGBA':
                    img = img.convert('RGB')

                # Mirror the image
                mirrored_img = img.transpose(Image.FLIP_LEFT_RIGHT)

                # Save the mirrored image to the destination directory
                mirrored_img.save(os.path.join(dest_dir, filename))


def copy_images(source_dirs, dest_dir):
    """
    Copy all images from the source directories to the destination directory.
    """
    # Create the destination directory if it doesn't exist
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    for source_dir in source_dirs:
        for filename in os.listdir(source_dir):
            if filename.endswith(".jpg"):
                src_path = os.path.join(source_dir, filename)
                dest_path = os.path.join(dest_dir, filename)
                # Copy the image
                shutil.copy(src_path, dest_path)

# Directories
left_hand_dir = "/content/drive/MyDrive/OptPhotoFiles/nonRA_Left_Hand"
right_hand_dir = "/content/drive/MyDrive/OptPhotoFiles/nonRA_Right_Hand"

left_hand_mirror_dir = "/content/drive/MyDrive/nonRA/nonRA_Left_Hand_mirror"
right_hand_mirror_dir = "/content/drive/MyDrive/nonRA/nonRA_Right_Hand_mirror"

opt_photo_ra_right_dir = "/content/drive/MyDrive/OptPhotoFiles/optPhotoNonRaRight"
opt_photo_ra_left_dir = "/content/drive/MyDrive/OptPhotoFiles/optPhotoNonRaLeft"

# Mirror images
mirror_images(left_hand_dir, left_hand_mirror_dir)
mirror_images(right_hand_dir, right_hand_mirror_dir)

# Copy images
copy_images([right_hand_dir, left_hand_mirror_dir], opt_photo_ra_right_dir)
copy_images([left_hand_dir, right_hand_mirror_dir], opt_photo_ra_left_dir)

"Images have been processed and saved to the specified directories."

'Images have been processed and saved to the specified directories.'

#欠番の同定
nonRAの画像は、画像の質がまちまちであるだけでなく、画像として読み込めない破損ファイルや、htmlファイルなどのもともと画像ではないファイルなども含まれていた。これらはこれまでの画像処理で削除されており、いくつかのファイル番号は欠番となっている。optPhotoTable_control上の欠番を削除し、tableと実際の画像ファイルの名称を完全に一致させるため、欠番の同定を行う。

In [3]:
import pandas as pd
import os

# CSVファイルと画像ディレクトリのパス
csv_file_path = '/content/drive/MyDrive/OptPhotoFiles/optPhotoTable_control.csv'
image_dir_path = '/content/drive/MyDrive/OptPhotoFiles/optPhotoNonRaRight'

# CSVファイルを読み込む
df = pd.read_csv(csv_file_path)

# ディレクトリ内の画像ファイル名のリストを取得
image_files = set(os.listdir(image_dir_path))

# CSVファイル内のファイル名のリストを取得
csv_files = set(df['filename'])

# ディレクトリには存在しないがCSVには存在するファイル名を特定
missing_files = csv_files - image_files

# 欠損ファイルのファイル名を表示
print("欠損しているファイル名:")
for file in missing_files:
    print(file)

# 欠損しているファイルの記録をCSVファイルから削除
df = df[~df['filename'].isin(missing_files)]

# CSVファイルを更新して保存
df.to_csv(csv_file_path, index=False)

"欠損ファイルの処理が完了しました。"

欠損しているファイル名:
nonRA_right_hand_232.jpg
nonRA_left_hand_1180.jpg
nonRA_right_hand_592.jpg
nonRA_left_hand_638.jpg
nonRA_right_hand_39.jpg
nonRA_left_hand_39.jpg
nonRA_left_hand_383.jpg
nonRA_right_hand_383.jpg
nonRA_right_hand_164.jpg
nonRA_left_hand_1188.jpg
nonRA_right_hand_651.jpg
nonRA_left_hand_571.jpg
nonRA_right_hand_357.jpg
nonRA_left_hand_1152.jpg
nonRA_right_hand_416.jpg
nonRA_left_hand_1151.jpg
nonRA_right_hand_241.jpg
nonRA_left_hand_547.jpg
nonRA_left_hand_841.jpg
nonRA_left_hand_399.jpg
nonRA_right_hand_352.jpg
nonRA_left_hand_21.jpg
nonRA_right_hand_1013.jpg
nonRA_left_hand_651.jpg
nonRA_right_hand_1151.jpg
nonRA_left_hand_382.jpg
nonRA_left_hand_74.jpg
nonRA_right_hand_399.jpg
nonRA_left_hand_993.jpg
nonRA_left_hand_592.jpg
nonRA_left_hand_501.jpg
nonRA_left_hand_357.jpg
nonRA_left_hand_164.jpg
nonRA_right_hand_841.jpg
nonRA_right_hand_1088.jpg
nonRA_left_hand_537.jpg
nonRA_left_hand_527.jpg
nonRA_left_hand_1178.jpg
nonRA_right_hand_74.jpg
nonRA_left_hand_416.jpg
nonRA_ri

'欠損ファイルの処理が完了しました。'